In [13]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import Ridge, RidgeCV, ElasticNet, LassoCV, LassoLarsCV
from sklearn.kernel_ridge import KernelRidge
from vecstack import stacking
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
import numpy as np

class Project:   
    def __init__(self):
        self.data = None
        self.train_data = None
        self.test_data = None
            
    def _load_data_(self,filepath):
      
        #if (~os.path.isfile('train.csv')) | (~os.path.isfile('test.csv')):
           # self.data = pd.read_csv(filepath,encoding='ISO-8859-1')
           # self.train_test_split()
           # self._write_data_()
            
        self.train_data = pd.read_csv('train.csv')
                                #,encoding='ISO-8859-1')
        self.test_data = pd.read_csv('test.csv',encoding='ISO-8859-1')
        return self.train_data
        
    def _write_data_(self):
        self.train_data.to_csv('train.csv')
        self.test_data.to_csv('test.csv')
    
    def train_test_split(self):
        self.train_data, self.test_data = train_test_split(self.data, test_size=0.2, random_state=42)   
        
    def null_summary(self,df):
        print(df.isnull().sum())
        all_data_na = (df.isnull().sum() / len(df)) * 100
        all_data_na = all_data_na.drop(all_data_na[all_data_na == 0].index).sort_values(ascending=False)
        missing_data = pd.DataFrame({'Missing Ratio' :all_data_na})
        missing_data.head()
        if missing_data.size> 0:
            f, ax = plt.subplots(figsize=(15, 12))
            plt.xticks(rotation='90')
            sns.barplot(x=all_data_na.index, y=all_data_na)
            plt.xlabel('Features', fontsize=15)
            plt.ylabel('Percent of missing values', fontsize=15)
            plt.title('Percent missing data by feature', fontsize=15)
            plt.show()
            
    def get_stats(self,df,variable):
        print("Skewness: %f" % df[variable].skew())
        print("Kurtosis: %f" % df[variable].kurt())
        print("Mean: %f" % df[variable].mean())
        print("Variance: %f" % (df[variable].var()))

            
    def correlation_map(self,df,outcome,k):
        corrmat = df.corr()
        col_large = corrmat.nlargest(k+1,outcome)[outcome].index
        col_small = corrmat.nsmallest(k,outcome)[outcome].index
        cols = col_large.union(col_small)
        cm = np.corrcoef(corrmat[cols].loc[cols].values.T)
        print(corrmat[cols].loc[outcome])
        #sns.set(font_scale=1.25)
        hm = sns.heatmap(cm, cbar=True, vmax=1,annot=True, fmt='.1f', annot_kws={'size': 8}, yticklabels=cols.values, xticklabels=cols.values)
        return cols
    
    def encode_organization(self,x):
            if 'Academy' in x:
                return 1
            elif 'Instituion' in x:
                return 2
            elif 'College' in x:
                return 3
            elif 'University' in x:
                return 4
            else:
                return 0
    
    def encode_cost(self,df):
        prices = []
        for i in range(len(df)):
            if pd.notnull(df['NPT4_PUB'][i]):
                prices.append(df['NPT4_PUB'][i])
            elif pd.notnull(df['NPT4_PRIV'][i]):
                prices.append(df['NPT4_PRIV'][i])
            else:
                prices.append(np.NaN) 
        
        df.loc[:,'NET_COST'] = pd.Series(prices, index=df.index)
        
        return df
    
    def encode_25KBinary(self, df):
        vals = []
        for i in range(len(df)):
            if float(df['gt_25k_p6'][i]) >= 0.6:
                vals.append(1)
            elif float(df['gt_25k_p6'][i]) < 0.6:
                vals.append(0)
            else:
                vals.append(np.NaN)
        
        df.loc[:,'Binary_25k'] = pd.Series(vals, index=df.index)
        
        return df
    
    def encode_state(self,x):
        west = ['WA','MT','OR','ID','WT','CA','NV','UT','CO','AZ','NM']
        midwest = ['ND','MN','SD','NE','KS','IA','MO','WI','IL','MI','IN','OH','WY']
        northeast = ['NY','PA','NJ','RI','CT','MA','VT','NH','ME']
        south = ['TX','OK','AR','LA','MS','AL','GA','TN','KY','WV','DC','MD','VA','DE','NC','SC','FL']
        others =['AK','HI','PR','PW','AS','GU','FM','VI']
        
        if x in others:
            return 0
        elif x in midwest:
            return 1
        elif x in south:
            return 2
        elif x in west:
            return 3
        elif x in northeast:
            return 4
        else: return x
        
    def compute_ci(self,x):
        print('tn,p,dof =',sms.DescrStatsW(x).ttest_mean(x.mean()))
        print('CI =',sms.DescrStatsW(x).tconfint_mean())
        
        

In [32]:
######Data Load##########

project = Project()
project._load_data_('train.csv')
df = project.train_data
df = df.drop(df.columns[0],axis=1)
df = df.apply(lambda x: x.replace('PrivacySuppressed',np.NAN))
df = project.encode_cost(df)
df = project.encode_25KBinary(df)
df['RPY_3YR_RT_SUPP'] = pd.to_numeric(df['RPY_3YR_RT_SUPP'])
df['md_earn_wne_p10'] = pd.to_numeric(df['md_earn_wne_p10'])
df['GRAD_DEBT_MDN_SUPP'] = pd.to_numeric(df['GRAD_DEBT_MDN_SUPP'])
df['GRAD_DEBT_MDN10YR_SUPP'] = pd.to_numeric(df['GRAD_DEBT_MDN10YR_SUPP'])
df['C150_4_POOLED_SUPP'] = pd.to_numeric(df['C150_4_POOLED_SUPP'])
df['C200_L4_POOLED_SUPP'] = pd.to_numeric(df['C200_L4_POOLED_SUPP'])
columns = df.columns
treatment_variables = columns.drop(['INSTNM','CITY','STABBR','RPY_3YR_RT_SUPP', 'md_earn_wne_p10', 'gt_25k_p6', 'Binary_25k'])
#only keep the variables if less than 20% of the values are missing ( SHOULD I KEEP IT at 20%?)
keep_variables = []
for i in range(len(treatment_variables)):
    x = df[str(treatment_variables[i])].isnull().sum()/float((len(df)))
    if x <= 0.2:
        keep_variables.append(treatment_variables[i]) 
target_var = 'RPY_3YR_RT_SUPP'
keep_variables.append(target_var)


In [33]:
df = df[keep_variables]
df = df.apply(lambda x: x.fillna(x.median()),axis=0)

train_X = df.loc[:, df.columns != target_var]
train_Y = df[target_var]

In [11]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import Ridge, RidgeCV, ElasticNet, LassoCV, LassoLarsCV,LinearRegression
from sklearn.kernel_ridge import KernelRidge
from vecstack import stacking
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold, learning_curve


models = [
#     ExtraTreesRegressor(random_state = 0, n_jobs = 2, 
#         n_estimators = 100, max_depth = 3),
    
#     Pipeline([
#       ('feature_selection', SelectFromModel(LinearSVC(penalty="l1"))),
#       ('regression',randomForestRegressor(random_state = 0, n_jobs = 2, 
#         n_estimators = 100, max_depth = 3))
#     ]),
    
#      make_pipeline(SelectFromModel(ExtraTreesRegressor(random_state = 0, n_jobs = 2, n_estimators = 500, max_depth = 3)),RandomForestRegressor(random_state = 0, n_jobs = 2, 
#          n_estimators = 100, max_depth = 3)),
        
#     lgb.LGBMRegressor(objective='regression',num_leaves=5,
#                               learning_rate=0.05, n_estimators=2000,
#                               max_bin = 60, bagging_fraction = 0.8,
#                               bagging_freq = 5, feature_fraction = 0.2319,
#                               feature_fraction_seed=9, bagging_seed=9,
#                               min_data_in_leaf =6, min_sum_hessian_in_leaf = 11),
    
    make_pipeline(RobustScaler(), LassoCV(alphas =[0.0005,0.001,0.0015], random_state=1,cv=3)),
    make_pipeline(RobustScaler(), ElasticNet(alpha=0.0005, l1_ratio=.9, random_state=3)),
    KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5),

    GradientBoostingRegressor(n_estimators=2000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5),
    
    ]



kfold = 5
# cv_results = []
# cv_means = []
# cv_std = []
# for model in models :
#     cv_results.append(cross_val_score(model, train_X, y = train_Y, scoring = "neg_mean_squared_error", cv = kfold, n_jobs=4))

# for cv_result in cv_results:
#     cv_means.append(cv_result.mean())
#     cv_std.append(cv_result.std())

# cv_res = pd.DataFrame({"CrossValMeans":cv_means,"CrossValerrors": cv_std,"Algorithm":["Lasso","ElasticNet","GradientBoosting"]})

# g = sns.barplot("CrossValMeans","Algorithm",data = cv_res, palette="Set3",orient = "h",**{'xerr':cv_std})
# g.set_xlabel("Mean Accuracy")
# g = g.set_title("Cross validation scores")


# Compute stacking features
#S_train, S_test = stacking(models, train_X.values, train_Y.values, test_X.values, 
#    regression = True, metric = mean_squared_error, n_folds = 20, 
#    shuffle = True, random_state = 0, verbose = 2)


#model = make_pipeline(RobustScaler(), ElasticNet(alpha=0.0001, l1_ratio=.9, random_state=3))

#model = RandomForestRegressor(random_state = 0, n_jobs = 2, n_estimators = 1000, max_depth = 3)
    
# Fit 2-nd level model
#model = model.fit(S_train, train_Y.values)

# Predict
#y_pred = model.predict(S_train)

# Final prediction score
#print('Final prediction score: [%.8f]' % mean_squared_error(train_Y.values, y_pred))
#y_pred = model.predict(S_test)

In [55]:
#ExtraTrees 
ExtC = ExtraTreesRegressor()
## Search grid for optimal parameters
ex_param_grid = {"max_depth": [5],
              "max_features": [1, 3, 5, 10],
              "min_samples_split": [2, 3, 10],
              "min_samples_leaf": [1, 3, 10],
              "bootstrap": [False],
              "n_estimators" :[100,200,500]
              }


gsExtC = GridSearchCV(ExtC,param_grid = ex_param_grid, cv=kfold, scoring="neg_mean_squared_error", n_jobs= 4, verbose = 1)

gsExtC.fit(train_X,train_Y)

ExtC_best = gsExtC.best_estimator_

# Best score
gsExtC.best_score_

Fitting 5 folds for each of 108 candidates, totalling 540 fits


[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    7.7s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:   37.3s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:  1.7min
[Parallel(n_jobs=4)]: Done 540 out of 540 | elapsed:  2.5min finished


-0.014204198042550534

In [48]:
RFC = RandomForestRegressor()


## Search grid for optimal parameters
rf_param_grid = {"max_depth": [5],
              "max_features": [1, 3, 5,10],
              #"min_samples_split": [2, 3, 10],
              #"min_samples_leaf": [1, 3, 10],
              #"bootstrap": [False],
              "n_estimators" :[100,200,500]}


gsRFC = GridSearchCV(RFC,param_grid = rf_param_grid, cv=kfold,scoring="neg_mean_squared_error", n_jobs= 4, verbose = 1)

gsRFC.fit(train_X,train_Y)

RFC_best = gsRFC.best_estimator_

# Best score
gsRFC.best_score_

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   21.2s
[Parallel(n_jobs=4)]: Done  60 out of  60 | elapsed:   41.9s finished


-0.012303051456565317

In [47]:
GBC = GradientBoostingRegressor()
gb_param_grid = {
              'n_estimators' : [100,200,500],
              'learning_rate': [0.05,0.02,0.01,0.005],
              'max_depth': [4, 8],
              'min_samples_leaf': [100,150],
              'max_features': [0.3, 0.1] 
              }

gsGBC = GridSearchCV(GBC,param_grid = gb_param_grid, cv=kfold,scoring="neg_mean_squared_error",n_jobs= 4, verbose = 1)

gsGBC.fit(train_X,train_Y)

GBC_best = gsGBC.best_estimator_

# Best score
gsGBC.best_score_

Fitting 5 folds for each of 96 candidates, totalling 480 fits


[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   14.4s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:  1.2min
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:  2.9min
[Parallel(n_jobs=4)]: Done 480 out of 480 | elapsed:  3.3min finished


-0.0096090257071771643

In [46]:
LR = LinearRegression()
parameters = {'fit_intercept':[True,False], 'normalize':[True,False], 'copy_X':[True, False]}
grid = GridSearchCV(LR,parameters, scoring="neg_mean_squared_error",cv=kfold)
grid.fit(train_X,train_Y)
LR_best = grid.best_estimator_
grid.best_score_

-0.016685755888763913

In [52]:
DTC = DecisionTreeRegressor()

adaDTC = AdaBoostRegressor(DTC, random_state=7)

ada_param_grid = {
              "base_estimator__splitter" :   ["best", "random"],
              "n_estimators" :[30,50],
              "learning_rate":  [0.0001, 0.001, 0.01, 0.1]}

gsadaDTC = GridSearchCV(adaDTC,param_grid = ada_param_grid, cv=kfold, scoring="neg_mean_squared_error", n_jobs= 4, verbose = 1)

gsadaDTC.fit(train_X,train_Y)

ada_best = gsadaDTC.best_estimator_
gsadaDTC.best_score_

Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   46.4s
[Parallel(n_jobs=4)]: Done  80 out of  80 | elapsed:   59.7s finished


-0.0096013012308369412